In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()

    return str_text

In [2]:
# read_file('moby_dick_four_chapters.txt')

In [3]:
# read_file('melville-moby_dick.txt')

In [4]:
import spacy

2023-09-09 18:25:37.455474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])

In [6]:
nlp.max_length = 1198623

In [7]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [8]:
d = read_file('moby_dick_four_chapters.txt')

In [9]:
tokens = separate_punc(d)

/opt/anaconda3/envs/nlp_course2/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [10]:
len(tokens)

11338

In [11]:
# 25 words --> network predict #26

In [12]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [13]:
type(text_sequences)

list

In [14]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [15]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [16]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [17]:
from keras.preprocessing.text import Tokenizer

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [19]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [20]:
# sequences[1]

In [21]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")
# tokenizer.index_word

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
314 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2713 : interest
14 : me
24 : on


In [22]:
# tokenizer.word_counts

In [23]:
vocabulary_size = len(tokenizer.word_counts)

In [24]:
vocabulary_size

2718

In [25]:
type(sequences)

list

In [26]:
import numpy as np

In [27]:
sequences = np.array(sequences)

In [28]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [29]:
from keras.utils import to_categorical

In [30]:
X = sequences[:,:-1]

In [31]:
y = sequences[:,-1]

In [32]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [33]:
seq_len = X.shape[1]

In [34]:
X.shape

(11312, 25)

In [35]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [36]:
def create_model(vocabulary_size, seq_len):

    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()

    return model

In [37]:
model = create_model(vocabulary_size+1, seq_len)

2023-09-09 18:26:49.677450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 50)            15200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 2719)              138669    
                                                                 
Total params: 244,594
Trainable params: 244,594
Non-trainable params: 0
_________________________________________________________________


In [38]:
from pickle import dump, load

In [39]:
model.fit(X, y, batch_size=128, epochs=2, verbose=1)

Epoch 1/2
89/89 [==============================] - 5s 28ms/step - loss: 6.9958 - accuracy: 0.0395
Epoch 2/2
89/89 [==============================] - 2s 28ms/step - loss: 6.3746 - accuracy: 0.0529


In [40]:
model.save('my_mobydick_model.h5')

In [41]:
dump(tokenizer, open('my_simpletokenizer', 'wb'))

In [43]:
from keras_preprocessing.sequence import pad_sequences

In [124]:
def generate_text(model: Sequential, tokenizer: Tokenizer, seq_len, seed_text, num_gen_words):

    output_text = []

    input_text = seed_text

    for i in range(num_gen_words):

        encoded_text = tokenizer.texts_to_sequences([input_text])[0]

        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        # pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0]
        pred_word_ind = np.argmax(pred_word_ind, axis=0)

        pred_word = tokenizer.index_word[pred_word_ind]

        input_text += ' ' + pred_word

        output_text.append(pred_word)

    return ' '.join(output_text)

In [125]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [126]:
import random
random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [127]:
random_seed_text = text_sequences[random_pick]

In [128]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [129]:
seed_text = ' '.join(random_seed_text)

In [130]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [131]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)

"to be seen there was no bad olfactories my own letter was cheerily listening over his hearers who 's more can go have a wearing"

In [132]:
from keras.models import load_model

In [133]:
model = load_model('epochBIG.h5')

In [134]:
tokenizer = load(open('epochBIG', 'rb'))

In [135]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)

"to be seen there was no bad olfactories my own letter was cheerily listening over his hearers who 's more can go have a wearing"